# Policy Impact Analysis
This notebook demonstrates how to read and analyze the 10-year budgetary impact data for policy reforms.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# Load the CSV data
df = pd.read_csv('policy_impacts.csv')
print(f"Data shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

In [ ]:
# Get summary statistics
print("Unique policies:")
policies = df[['policy_id', 'policy_name']].drop_duplicates()
for _, row in policies.iterrows():
    print(f"  {row['policy_id']}: {row['policy_name']}")

print(f"\nTotal number of records: {len(df)}")
print(f"Year range: {df['year'].min()} - {df['year'].max()}")

In [ ]:
# Calculate 10-year total impact for each policy
def calculate_total_impact(policy_id, credit_value=None):
    if credit_value is not None:
        mask = (df['policy_id'] == policy_id) & (df['credit_value'] == credit_value)
    else:
        mask = (df['policy_id'] == policy_id) & (df['credit_value'].isna())
    return df[mask]['impact_billions'].sum()

# Calculate totals for all policies
totals = []
for policy_id in df['policy_id'].unique():
    policy_name = df[df['policy_id'] == policy_id]['policy_name'].iloc[0]
    
    if policy_id == 4:  # Child Tax Credit has multiple values
        for credit_val in [300, 600, 900, 1200, 1500]:
            total = calculate_total_impact(policy_id, credit_val)
            totals.append({
                'Policy': f"{policy_name} (${credit_val})",
                'Total Impact (Billions)': total
            })
    else:
        total = calculate_total_impact(policy_id)
        totals.append({
            'Policy': policy_name,
            'Total Impact (Billions)': total
        })

totals_df = pd.DataFrame(totals)
totals_df = totals_df.sort_values('Total Impact (Billions)')
print("10-Year Total Impacts:")
totals_df

In [ ]:
# Visualize year-by-year impacts for all policies (excluding Child Tax Credit variations)
fig, ax = plt.subplots(figsize=(14, 8))

# Plot each policy except Child Tax Credit
for policy_id in [1, 2, 3, 5, 6]:
    policy_data = df[(df['policy_id'] == policy_id) & (df['credit_value'].isna())]
    ax.plot(policy_data['year'], 
            policy_data['impact_billions'], 
            marker='o', 
            linewidth=2.5,
            label=policy_data['policy_name'].iloc[0])

# Plot Child Tax Credit with $900 as representative
ctc_data = df[(df['policy_id'] == 4) & (df['credit_value'] == 900)]
ax.plot(ctc_data['year'], 
        ctc_data['impact_billions'], 
        marker='o', 
        linewidth=2.5,
        label='Child Tax Credit ($900)')

ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax.set_xlabel('Fiscal Year', fontsize=12)
ax.set_ylabel('Budget Impact (Billions $)', fontsize=12)
ax.set_title('10-Year Budgetary Impact by Policy Reform', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Compare Child Tax Credit variations
fig, ax = plt.subplots(figsize=(14, 8))

colors = plt.cm.coolwarm(np.linspace(0.2, 0.8, 5))
credit_values = [300, 600, 900, 1200, 1500]

for i, credit_val in enumerate(credit_values):
    ctc_data = df[(df['policy_id'] == 4) & (df['credit_value'] == credit_val)]
    ax.plot(ctc_data['year'], 
            ctc_data['impact_billions'], 
            marker='o', 
            linewidth=2.5,
            color=colors[i],
            label=f'${credit_val} Credit')

ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax.set_xlabel('Fiscal Year', fontsize=12)
ax.set_ylabel('Budget Impact (Billions $)', fontsize=12)
ax.set_title('Child Tax Credit: Impact by Credit Value', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create a heatmap showing impact by year and policy
# Prepare data for heatmap (using representative values)
heatmap_data = []
for policy_id in [1, 2, 3, 5, 6]:
    policy_data = df[(df['policy_id'] == policy_id) & (df['credit_value'].isna())]
    for _, row in policy_data.iterrows():
        heatmap_data.append({
            'Policy': row['policy_name'],
            'Year': row['year'],
            'Impact': row['impact_billions']
        })

# Add Child Tax Credit with $900 value
ctc_data = df[(df['policy_id'] == 4) & (df['credit_value'] == 900)]
for _, row in ctc_data.iterrows():
    heatmap_data.append({
        'Policy': 'Child Tax Credit ($900)',
        'Year': row['year'],
        'Impact': row['impact_billions']
    })

heatmap_df = pd.DataFrame(heatmap_data)
pivot_table = heatmap_df.pivot(index='Policy', columns='Year', values='Impact')

fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(pivot_table, 
            annot=True, 
            fmt='.0f', 
            cmap='RdYlGn_r', 
            center=0,
            cbar_kws={'label': 'Impact (Billions $)'},
            linewidths=0.5,
            ax=ax)
ax.set_title('Policy Impact Heatmap (10-Year Budget Impact)', fontsize=14, fontweight='bold')
ax.set_xlabel('Fiscal Year', fontsize=12)
ax.set_ylabel('Policy', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Export data for React app consumption
# This cell demonstrates how to prepare data for the React app

def prepare_react_data():
    """Prepare data in format suitable for React app"""
    react_data = {}
    
    for policy_id in df['policy_id'].unique():
        policy_name = df[df['policy_id'] == policy_id]['policy_name'].iloc[0]
        
        if policy_id == 4:  # Child Tax Credit
            react_data[f'policy_{policy_id}'] = {}
            for credit_val in [300, 600, 900, 1200, 1500]:
                data = df[(df['policy_id'] == policy_id) & (df['credit_value'] == credit_val)]
                react_data[f'policy_{policy_id}'][f'credit_{credit_val}'] = [
                    {'year': int(row['year'] - 2024), 'impact': row['impact_billions']} 
                    for _, row in data.iterrows()
                ]
        else:
            data = df[(df['policy_id'] == policy_id) & (df['credit_value'].isna())]
            react_data[f'policy_{policy_id}'] = [
                {'year': int(row['year'] - 2024), 'impact': row['impact_billions']} 
                for _, row in data.iterrows()
            ]
    
    return react_data

react_data = prepare_react_data()
print("Data structure for React app:")
print(f"Keys: {list(react_data.keys())}")
print(f"\nExample (Policy 1 first 3 years): {react_data['policy_1'][:3]}")